# Example 4: Optimize a noisy multivariate system

Bayesian optimization also works for stochastic response functions. Here we illustrate this for a noisy multivariate system by determining the maximal value.

## Framework approach

We will solve the problem in two different ways:
1. using the closed-loop approach of the `.auto`-method
2. using the iterative optimization approach of the framework which requires using the methods `.ask` and `.tell`. This approach allows for iterative optimization and optimization of any callable function, known or otherwise.

The optimization process can be stopped after any number of iterations.


## Technical note

Installation of `torch` and `torchvision` (required dependencies) cannot be bundled as part of the `creative_project` installable. This is unfortunate, but a known issue it seems. Therefore these must be installed first, before installing `creative_project`.

### Get started: Import libraries

In [ ]:
# Preamble

# Install torch and torchvision. Use this link to identify the right versions to install on your system 
# depending on configuration: https://pytorch.org/get-started/locally/
#
#pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
#
# Install creative_project from github (will require authentication with password)
#pip install --user https://github.com/svedel/kre8_core/
! pip install --user greattunes

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from greattunes import TuneSession
import pandas as pd
%matplotlib inline

### Set up the problem to be solved

Here create a noisy multivariate function defined below for the input vector $\mathbf{x} = (x_0, x_1)$

$$
f(\mathbf{x}) = - \frac{ (6 x_0 -2)^2 (6 x_1 -2)^2 \, \sin{(12 x_0- 4)} \sin{(12 x_1 - 4)} }{250} + \frac{1}{2 \sigma^2 \pi} \mathrm{e}^{- \left(\frac{x_0 - 0.5}{\sigma} \right)^2 - \left( \frac{x_1 - 0.5}{\sigma} \right)^2 }   + \xi \quad , \quad x_i \in [0; 1], \ i=0,1
$$

where $\xi$ is random number drawn from a uniform distribution (range $[0; 1]$) and $\sigma = 0.1$. This function has its average global maximum at $\mathbf{x}^* = (0.5,0.5)$.


In [ ]:
# define the function
def f2_dup(x):
    covar0, covar1 = np.meshgrid(x["covar0"].values, x["covar1"].values)
    sigma = 0.1
    func_raw = (-(6 * covar0 - 2) ** 2 * np.sin(12 * covar0 - 4))*(-(6 * covar1 - 2) ** 2 * np.sin(12 * covar1 - 4))/250 + 1/np.sqrt(2*sigma**2*np.pi) * np.exp(-((covar0-0.5)/sigma)**2 - ((covar1-0.5)/sigma)**2 )
    noise = torch.rand(covar0.shape).numpy()
    return  np.add(func_raw, noise)

Plot the response function

In [ ]:
# create an easy way to generate a surface plot of the function
def f2_dup_plot(x_vec):
    x0, x1 = np.meshgrid(x_vec, x_vec)
    output = f2_dup(pd.DataFrame({"covar0": x_vec, "covar1": x_vec}))
    return x0, x1, output

# generate the data for the figure
x = np.linspace(0,1)
x0, x1, output = f2_dup_plot(x)

fig = plt.figure(figsize=(8, 8))
ax = fig.gca(projection='3d')

surf = ax.plot_surface(x0, x1, output)
ax.set_xlabel("Covariate covar_0")
ax.set_ylabel("Covariate covar_1")
ax.set_zlabel("Response f_2")
plt.show()

Define the range for the covariates

In [ ]:
# define the range of interest
x0_init = 0.2
x1_init = 0.8
covars2d = [(x0_init, 0, 1), (x1_init, 0, 1)]

### Solution 1: Closed-loop solution approach using `.auto` method

Instantiate the `TuneSession` class and solve the problem

In [ ]:
# initialize class instance
cc = TuneSession(covars=covars2d)

# number of iterations
max_iter = 20

# run the auto-method
cc.auto(response_samp_func=f2_dup, max_iter=max_iter)

**PLOT THE PATH TO OPTIMALITY**

The best solution

In [ ]:
# run current_best method
cc.current_best()

### Solution 2: Iterative solution using `.ask` and `.tell` methods

Instantiate the `TuneSession` class and solve the problem. In this case, we need to write our own loop to iterate. Notice that the `covars` and `response` variables are converted to `torch` tensors of size $1 \times \mathrm{\#covariates}$ to store them in the instantiated class, where they are used for retraining the model at each iteration.

In [ ]:
from greattunes.data_format_mappings import tensor2pretty_covariate

# initialize the class instance
cc2 = TuneSession(covars=covars2d)

# run the solution
for i in range(max_iter):
    # generate candidate
    cc2.ask()

    # sample response
    # tensor2pretty_covariate maps between the backend dataformat used by 'proposed_X' to the pandas-based format consumed by the response function
    # the attribute 'covar_details' keeps information that maps backend and pandas ("pretty") dataformats
    covars = tensor2pretty_covariate(train_X_sample=cc2.proposed_X[-1].reshape(1,2), covar_details=cc2.covar_details)
    response = pd.DataFrame({"Response": [f2_dup(covars)]})

    # report response
    cc2.tell(covar_obs=covars, response_obs=response)

Best guess after solving

In [ ]:
# run current_best method
cc2.current_best()